In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1. Gerekli Kütüphanelerin Yüklenmesi**

İlk adımda, proje için gerekli olan Python kütüphanelerini yükledim. Bu kütüphaneler, veri işlemeden model geliştirmeye kadar geniş bir kullanım alanı sağlıyor.

Bu adımda şu kütüphaneler kullanılmaktadır:

Numpy ve Pandas: Veri işleme ve veri çerçevesi yönetimi.
Matplotlib ve Seaborn: Görselleştirme araçları.
OpenCV (cv2): Görüntü işleme kütüphanesi.
Scikit-learn: Model değerlendirme metrikleri ve veri ayırma.
Keras/TensorFlow: Derin öğrenme modelinin oluşturulması ve eğitimi için kullanılan kütüphaneler.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import warnings
warnings.filterwarnings("ignore")

**2. Veri Setinin Hazırlanması**

Veri seti, balık türlerini içeren resimlerden oluşuyor. Bu veri setini işlemeye başlamadan önce, her resmin yolunu ve sınıfını elde etmem gerekiyordu. 
Bunu aşağıdaki kodla yaptım:

In [ ]:
# Veri seti konumunu belirtin
dataset_path = "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"

# Sınıfları alın
fish_classes = [folder for folder in os.listdir(dataset_path) if '.' not in folder]

# Resim yolları ve sınıflarını depolamak için listeler
image_labels = []
image_paths = []

for current_dir, _, filenames in os.walk(dataset_path):
    if current_dir.split() and current_dir.split()[-1] != 'GT':
        for filename in filenames:
            if os.path.splitext(filename)[-1].lower() == ".png":
                image_labels.append(os.path.split(current_dir)[-1])
                image_paths.append(os.path.join(current_dir, filename))

# Veri setini bir DataFrame'e dönüştürün
data = pd.DataFrame({'path': image_paths, 'class': image_labels})

# Sınıfları görüntüleyin
print(data['class'].value_counts())


*Bu adımda:*

*Tüm resimlerin yollarını ve sınıflarını topladım.
Bu verileri bir Pandas DataFrame formatında depoladım.*

**3. Veri Dağılımının Görselleştirilmesi**

Veri setindeki balık sınıflarının dengeli olup olmadığını kontrol etmek amacıyla, her sınıftan kaç adet örnek olduğunu görselleştirdim.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='class', order=data['class'].value_counts().index, palette='viridis')
plt.title('Number of Samples per Fish Class')
plt.xlabel('Fish Class')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

> Bu adım, veri setinin sınıf dengesini anlamama yardımcı oldu. Her balık türünden eşit sayıda örnek olduğunu doğruladım,
> bu da sınıflandırma modelimi dengeli bir şekilde eğitmem için olumlu bir faktör.

**4. Örnek Resimlerin Görüntülenmesi**

Veri setinde bulunan balık türlerinden birer örnek görüntü alarak veri setinin içeriğini daha iyi anlamak istedim.

In [ ]:
# Her sınıftan bir örnek resmi göster
plt.figure(figsize=(20, 20))
for idx, unique_label in enumerate(data['class'].unique()):
    plt.subplot(3, 3, idx + 1)
    img = plt.imread(data[data["class"] == unique_label].iloc[0, 0])
    plt.imshow(img)
    plt.title(unique_label)
    plt.axis("off")
plt.show()

*Her balık sınıfından birer örnek resim görüntüleyerek, görsellerin kalitesi ve çeşitliliği hakkında bilgi sahibi oldum.*

**5. Resim Boyutlarının Belirlenmesi**

Tüm resimler aynı boyutta olmalı ki modelde tutarlı bir şekilde işlenebilsin. Bu yüzden resim boyutlarını 128x128 olarak ayarladım.

In [ ]:
# Resim boyutlarını belirleyin
img_shape = (128, 128, 3)

**6. Veriyi Eğitim ve Test Setlerine Ayırma**

Veriyi eğitim, doğrulama ve test setlerine ayırdım. Böylece modelin eğitimi sırasında veri setinin bir kısmını doğrulama için kullanarak modelin başarısını daha iyi değerlendirebildim.

In [ ]:
def get_generators(data, is_rgb=True):
    color_mode = 'rgb' if is_rgb else 'grayscale'
    data_full, data_remainder = train_test_split(data, stratify=data['class'], test_size=0.2)
    
    train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.1)
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    
    train_generator = train_datagen.flow_from_dataframe(
        data_full, x_col='path', y_col='class',
        target_size=img_shape[:-1], color_mode=color_mode,
        class_mode='categorical', batch_size=32, shuffle=True, subset='training'
    )
    val_generator = train_datagen.flow_from_dataframe(
        data_full, x_col='path', y_col='class',
        target_size=img_shape[:-1], color_mode=color_mode,
        class_mode='categorical', batch_size=32, shuffle=True, subset='validation'
    )
    test_generator = test_datagen.flow_from_dataframe(
        data_remainder, x_col='path', y_col='class',
        target_size=img_shape[:-1], color_mode=color_mode,
        class_mode='categorical'
    )
    return train_generator, val_generator, test_generator

train_gen, val_gen, test_gen = get_generators(data)

*Bu adımda:*

*Veriyi eğitim (%80), doğrulama ve test (%20) setlerine ayırdım.
Eğitim sırasında doğrulama verisiyle modelin genel performansını izledim.*

**7. Modelin Oluşturulması**

Bir Convolutional Neural Network (CNN) modeli oluşturdum. Modelde evrişim ve pooling katmanları kullanarak balık resimlerinden özellik çıkarımı yapıldı.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

# Modeli oluşturma
model = Sequential([
    Conv2D(16, (3, 3), strides=1, padding='same', input_shape=img_shape, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(fish_classes), activation='softmax')
])

*Bu adımda, modelin katman yapısı şu şekilde oluşturuldu:*

*4 adet evrişim katmanı ve ardından max-pooling katmanları.
Tam bağlı (dense) katman ile sınıflandırma yapıldı.*

**8. Modelin Derlenmesi ve Eğitilmesi**

Modeli derleyip eğitmeye başladım. Modelin aşırı öğrenmesini engellemek için early stopping kullanarak doğrulama verisinin performansını izledim.

In [ ]:
schedule = ExponentialDecay(initial_learning_rate=0.002, decay_steps=1000, decay_rate=0.7)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=schedule), metrics=['accuracy'])

earlystop = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(train_gen, validation_data=val_gen, epochs=100, callbacks=[earlystop])

*Bu adımda:*

*Model, Adam optimizasyon algoritması ve categorical_crossentropy kayıp fonksiyonu ile derlendi.
EarlyStopping kullanılarak aşırı öğrenme engellendi ve modelin en iyi hali geri yüklendi.*

**9. Model Performansının Görselleştirilmesi**

Eğitim ve doğrulama kayıplarını ve doğruluk oranlarını grafiksel olarak inceledim.

In [ ]:
plt.figure(figsize=(14, 5))

# Kayıp grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Kayıp Grafiği')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()

# Doğruluk grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Doğruluk Grafiği')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()

plt.tight_layout()
plt.show()


*Eğitim ve doğrulama kayıplarını ve doğruluk değerlerini analiz ederek modelin zamanla nasıl iyileştiğini inceledim.*

**10. Test Seti Performansının Değerlendirilmesi**

Modeli test verisi üzerinde değerlendirdim ve karışıklık matrisi ile sınıflandırma raporu oluşturdum.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_gen)
print(f'Test Kaybı: {test_loss:.4f}, Test Doğruluğu: {test_accuracy:.4f}')


In [ ]:
y_true = test_gen.classes
y_pred = model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=test_gen.class_indices.keys(), 
            yticklabels=test_gen.class_indices.keys())
plt.title('Karışıklık Matrisi')
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')
plt.show()

print(classification_report(y_true, y_pred_classes, target_names=test_gen.class_indices.keys()))

*Son olarak, test seti üzerindeki doğruluk oranını ve kaybı hesapladım. Karışıklık matrisi oluşturarak her sınıfın tahmin doğruluğunu analiz ettim.*